In [1]:
output_data = "data"

In [2]:
import yaml
# find which classes need to be removed and which need to be remapped
# for both datasets

def swap_keys(dict_):
    return dict((v,k) for k,v in dict_.items())

data = "coco128"
data_yaml_path1 = "yolov5/data/VisDrone.yaml"
data_yaml_path2 = "yolov5/data/VOC.yaml"

with open(data_yaml_path1, "r") as stream:
    data_yaml1 = yaml.safe_load(stream)

with open(data_yaml_path2, "r") as stream:
    data_yaml2 = yaml.safe_load(stream)

dict1 = swap_keys(data_yaml1["names"])
dict2 = swap_keys(data_yaml2["names"])

remove_dict = {x for x in dict1 if x in dict2}
remove_from_dataset1 = set()
remove_from_dataset2 = set()

for i in remove_dict:
    if i in dict1:
        remove_from_dataset1.add(dict1[i])
        del dict1[i]
    if i in dict2:
        remove_from_dataset2.add(dict2[i])
        del dict2[i]

remap_dataset1 = {}
remap_dataset2 = {}
count = 0
final_names = {}

for i in dict1:
    nr = dict1[i]
    remap_dataset1[nr] = count
    final_names[count] = i
    count += 1

for i in dict2:
    nr = dict2[i]
    remap_dataset2[nr] = count
    final_names[count] = i
    count += 1


In [11]:
import shutil
# create dataset1
import os
dataset1_path = "datasets/VisDrone/VisDrone2019-DET-train"
output_path1 = "data/images1"

cache = set()
for i in os.listdir(os.path.join(dataset1_path, "images")):
    name, _ = i.split(".")
    cache.add(name)

    shutil.copy2(os.path.join(dataset1_path, "images", i), os.path.join(output_path1, i))

    if len(cache) == 1000:
        break
for i in os.listdir(os.path.join(dataset1_path, "labels")):
    name, _ = i.split(".")
    if name in cache:
        text = ""
        with open(os.path.join(dataset1_path, "labels", i), "r") as f:
            for line in f.readlines():
                nr, a, b, c, d = line.strip().split(" ")
                nr = int(nr)
                if nr in remove_from_dataset1:
                    continue
                new_nr = remap_dataset1[nr]
                new_line = f"{new_nr} {a} {b} {c} {d}\n"
                text += new_line

        with open(f"data/labels1/{i}", "w") as text_file:
            text_file.write(text)

In [14]:
# create dataset2

dataset1_path = "datasets/VOC"
output_path1 = "data/images2"

cache = set()
for i in os.listdir(os.path.join(dataset1_path, "images", "train2007")):
    name, _ = i.split(".")
    cache.add(name)

    shutil.copy2(os.path.join(dataset1_path, "images", "train2007", i), os.path.join(output_path1, i))

    if len(cache) == 1000:
        print("1000")
        break
for i in os.listdir(os.path.join(dataset1_path, "labels", "train2007")):
    name, _ = i.split(".")
    if name in cache:
        text = ""
        with open(os.path.join(dataset1_path, "labels", "train2007", i), "r") as f:
            for line in f.readlines():
                nr, a, b, c, d = line.strip().split(" ")
                nr = int(nr)
                if nr in remove_from_dataset2:
                    continue
                new_nr = remap_dataset2[nr]
                new_line = f"{new_nr} {a} {b} {c} {d}\n"
                text += new_line

        with open(f"data/labels2/{i}", "w") as text_file:
            text_file.write(text)


1000


In [7]:
for k, v in final_names.items():
    print(f"{k}: '{v}',")

NameError: name 'final_names' is not defined

In [17]:
# make a test set from data

import os

files = set()
for i in os.listdir("data/dataset1/train/images"):
    name = i.split(".")[0]
    files.add(name)
    os.rename(f"data/dataset1/train/images/{i}", f"data/dataset1/test/images/{i}")
    if len(files) == 250:
        break

for i in os.listdir("data/dataset1/train/labels"):
    name = i.split(".")[0]
    if name not in files:
        continue
    os.rename(f"data/dataset1/train/labels/{i}", f"data/dataset1/test/labels/{i}")


In [19]:
files = set()
for i in os.listdir("data/dataset2/train/images"):
    name = i.split(".")[0]
    files.add(name)
    os.rename(f"data/dataset2/train/images/{i}", f"data/dataset2/test/images/{i}")
    if len(files) == 250:
        break

for i in os.listdir("data/dataset2/train/labels"):
    name = i.split(".")[0]
    if name not in files:
        continue
    os.rename(f"data/dataset2/train/labels/{i}", f"data/dataset2/test/labels/{i}")

datasets are made

now train them with yolo

use colab if possible

In [5]:
# validate model

import yolov5.val as v

opt = v.parse_opt(known=True)
setattr(opt, "data", "data/data_test.yaml")
setattr(opt, "weights", "100.pt")
setattr(opt, "task", "test")
setattr(opt, "save_txt", True)
setattr(opt, "save_hybrid", True)
setattr(opt, "save_conf", True)
setattr(opt, "verbose", True)
v.main(opt)

val: data=yolov5/data/coco128.yaml, weights=yolov5/yolov5s.pt, batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
WARNING ⚠️ --save-hybrid will return high mAP from hybrid labels, not from predictions alone
YOLOv5 🚀 2023-2-1 Python-3.8.16 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4040MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7074853 parameters, 0 gradients, 16.0 GFLOPs
test: Scanning /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v4/data/dataset1/test/labels... 500 images, 35 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<00:00, 1828.42it/s]
test: New cache created: /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v4/data/dataset1/test/labels.cache
                 Class     Images  Instances          P  

train new model with colab using old weights

In [6]:
# validate model

import yolov5.val as v

opt = v.parse_opt(known=True)
setattr(opt, "data", "data/data_test.yaml")
setattr(opt, "weights", "100_100.pt")
setattr(opt, "task", "test")
setattr(opt, "save_txt", True)
setattr(opt, "save_hybrid", True)
setattr(opt, "save_conf", True)
setattr(opt, "verbose", True)
v.main(opt)

val: data=yolov5/data/coco128.yaml, weights=yolov5/yolov5s.pt, batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=yolov5/runs/val, name=exp, exist_ok=False, half=False, dnn=False
WARNING ⚠️ --save-hybrid will return high mAP from hybrid labels, not from predictions alone
YOLOv5 🚀 2023-2-1 Python-3.8.16 torch-1.13.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4040MiB)

Fusing layers... 
Model summary: 157 layers, 7074853 parameters, 0 gradients, 16.0 GFLOPs
test: Scanning /home/erik/PycharmProjects/dynamic-yolo/app/prototype/v4/data/dataset1/test/labels.cache... 500 images, 35 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95:  19%|█▉        | 3/16 [00:18<01:26,  6.68s/it]WARNING ⚠️ NMS time limit 2.100s exceeded